In [231]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

import sklearn.metrics as metrics
from sklearn.model_selection import learning_curve

### Cargamos los archivos con los que vamos a trabajar.

In [232]:
#Leemos el archivo de entrenamiento 
df=pd.read_excel('E-Commerce_train.xlsx')


In [233]:
#Leemos el archivo de testeo
test=pd.read_excel('E-Commerce_test.xlsx')

Instanciamos X para los campos a analizar, Y para nuestro campo objetivo

In [234]:
X=df
X.rename({'Reached.on.Time_Y.N':'target'}, axis=1, inplace=True)
y=X['target']
X=X.drop(columns=['target'])

Borramos las columnas que no aportan a nuestro modelo

In [235]:
X=pd.concat([X,test])
X=X.drop(columns=['ID','Gender'])

## Codificamos para quitar strings

In [236]:
from sklearn.preprocessing import LabelEncoder

#Shipment (Flight=0, road=1 o ship=2)

Shipment=X['Mode_of_Shipment'].values
enc=LabelEncoder().fit(Shipment)

Shipment=enc.transform(Shipment)
Shipment=pd.DataFrame(Shipment)

X.drop(columns="Mode_of_Shipment", inplace=True)
X.insert(1,'Mode_of_Shipment', Shipment)


#Warehouse_block (D=3,F=4,A=0,B=1,C=2)

Warehouse_block=X['Warehouse_block'].values
enc=LabelEncoder().fit(Warehouse_block)

Warehouse_block=enc.transform(Warehouse_block)
Warehouse_block=pd.DataFrame(Warehouse_block)

X.drop(columns="Warehouse_block", inplace=True)
X.insert(0,'Warehouse_block', Warehouse_block)


#Product_importance (low=1, medium=2, high=0)

Product_importance=X['Product_importance'].values
enc=LabelEncoder().fit(Product_importance)

Product_importance=enc.transform(Product_importance)
Product_importance=pd.DataFrame(Product_importance)

X.drop(columns="Product_importance", inplace=True)
X.insert(6,'Product_importance', Product_importance)

In [219]:
X

,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Discount_offered,Weight_in_gms
0,3,0,4,2,177,3,1,44,1233
1,4,0,4,5,216,2,1,59,3088
2,0,0,2,2,183,4,1,48,3374
3,1,0,3,3,176,4,2,10,1177
4,2,0,2,2,184,3,2,46,2484
...,...,...,...,...,...,...,...,...,...
1995,0,2,4,1,252,5,2,1,1538
1996,1,2,4,1,232,5,2,6,1247
1997,2,2,5,4,242,5,1,4,1155
1998,4,2,5,2,223,6,2,2,1210


### Escalamos (normalizamos).

`Sólo vamos a tomar los campos que tienen valores mas altos.`

In [237]:
#Solo vamos a escalar 3 campos.
columnas_testeo = ['Cost_of_the_Product','Discount_offered', 'Weight_in_gms']
X_ajustar = X[columnas_testeo]

In [238]:
# Normalizamos con MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X_ajustar) # Scale the features

X_scaled=pd.DataFrame(X_scaled)

X_scaled.rename({0:'Cost_of_the_Product',1:'Discount_offered',2:'Weight_in_gms'}, axis=1, inplace=True)

Luego sustituimos los campos normalizados, con en nuestro df

In [239]:
#Concatenamos las celdas normalizadas con el resto del df
X1=X_scaled
X1=X1.reset_index(drop=True)
X2=X.drop(columns=['Cost_of_the_Product','Discount_offered', 'Weight_in_gms'])
X2=X2.reset_index(drop=True)


X=pd.concat([X1,X2],axis=1)

In [240]:
X

,Cost_of_the_Product,Discount_offered,Weight_in_gms,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Prior_purchases,Product_importance
0,0.378505,0.671875,0.033893,3,0,4,2,3,1
1,0.560748,0.906250,0.304894,4,0,4,5,2,1
2,0.406542,0.734375,0.346676,0,0,2,2,4,1
3,0.373832,0.140625,0.025712,1,0,3,3,4,2
4,0.411215,0.703125,0.216654,2,0,2,2,3,2
...,...,...,...,...,...,...,...,...,...
10994,0.728972,0.000000,0.078451,0,2,4,1,5,2
10995,0.635514,0.078125,0.035939,1,2,4,1,5,2
10996,0.682243,0.046875,0.022498,2,2,5,4,5,1
10997,0.593458,0.015625,0.030533,4,2,5,2,6,2


### Separamos en el archivo de entrenamiento y testeo original

`Debemos cuidar que queden del mismo tamaño original`

In [241]:
test=X.iloc[8999:,:]
X=X.iloc[:8999,:]


In [242]:
X.shape

(8999, 9)

In [243]:
y.shape

(8999,)

In [244]:
test.shape

(2000, 9)

# Finalmente hacemos un balanceo con over sampling

In [245]:
#Esto es para over_sampling

from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X, y.ravel())
print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))


After OverSampling, the shape of train_X: (11438, 9)
After OverSampling, the shape of train_y: (11438,) 

After OverSampling, counts of label '1': 5719
After OverSampling, counts of label '0': 5719


### `Usemos Random Forest`

In [29]:
# columnas_testeo = ['Discount_offered',  'Weight_in_gms']
# test1 = test[columnas_testeo]

In [246]:
from sklearn.ensemble import RandomForestClassifier

# Instanciamos el modelo

clf = RandomForestClassifier(n_estimators=50, max_features="log2", n_jobs=-1, oob_score = True, random_state = 100)
clf.fit(X_train_res,y_train_res)

# Evaluamos

y_train_pred = clf.predict(X_train_res)
y_test_pred = clf.predict(test)




In [247]:
print(metrics.accuracy_score(y_train_res, y_train_pred))

1.0


In [248]:
prueba=pd.DataFrame(y_test_pred)
prueba.rename({0:'pred'}, axis=1, inplace=True)
prueba.to_csv('Jhovanylara.csv')

In [249]:
prueba.shape

(2000, 1)